In [38]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import sys
import os

sys.path.append(os.path.abspath("/data/home/ngjhn/read"))


In [ ]:
from coco import *
from adapter_utils import *
from utils import *

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


In [ ]:
from transformers import VisionEncoderDecoderModel
from transformers.adapters import LoRAConfig, BertAdapterModel, ModelAdaptersMixin, ModelWithFlexibleHeadsAdaptersMixin



config = LoRAConfig(r=8)
text_model = BertAdapterModel.from_pretrained("bert-base-uncased")
# freeze(text_model)
model = VisionEncoderDecoderModel.from_encoder_decoder_pretrained(
    "google/vit-base-patch16-224-in21k", "bert-base-uncased")
# freeze(model)


# for p in model.parameters():
#     p.requires_grad = False
text_model.add_adapter("lora_adapter", config=config)
# text_model.add_causal_lm_head("lm_head")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertAdapterModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertAdapterModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertAdapterModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.weig

In [ ]:
text_model

BertAdapterModel(
  (shared_parameters): ModuleDict()
  (bert): BertModel(
    (shared_parameters): ModuleDict()
    (invertible_adapters): ModuleDict()
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(
                in_features=768, out_features=768, bias=True
                (loras): ModuleDict(
                  (lora_adapter): LoRA()
                )
              )
              (key): Linear(
                in_features=768, out_features=768, bias=True
                (loras): ModuleDict()
              )
              (

In [ ]:
from copy import deepcopy
from copy import deepcopy

for layer, adapter_layer in zip(model.decoder.bert.encoder.layer, text_model.bert.encoder.layer):
    layer.attention.self.query.loras = deepcopy(adapter_layer.attention.self.query.loras)
    # layer.attention.self.query.loras.lora_adapter
    layer.attention.self.key.loras = deepcopy(adapter_layer.attention.self.key.loras)
    layer.attention.self.value.loras = deepcopy(adapter_layer.attention.self.value.loras)
    # layer.attention.self.value.loras.lora_adapter
    layer.intermediate.dense.loras = deepcopy(adapter_layer.intermediate.dense.loras)
    layer.output.dense.loras = deepcopy(adapter_layer.output.dense.loras)

In [ ]:

data_module = CocoDataModule(
    data_dir="/data/home/ngjhn/read/coco",
    ann_dir="/data/home/ngjhn/read/coco/annotations/",
    vision_model_name_or_path="google/vit-base-patch16-224-in21k",
    split_file="/data/home/ngjhn/read/coco/dataset_coco.json",
    collate_fn=collate_fn,
    train_batch_size=32,
    val_batch_size=128)

pl_model = ImageCaptioningModel(
    "google/vit-base-patch16-224-in21k",
    "bert-base-uncased",
    data_module=data_module,
    learning_rate=1e04,
    peft_conf=FTConf(ft_type="full"),
    model=model,
)


Enables full tuning; num trainable params 138,176,058
loading annotations into memory...
Done (t=0.56s)
creating index...
index created!
loading annotations into memory...
Done (t=0.26s)
creating index...
index created!
loading annotations into memory...
Done (t=0.25s)
creating index...
index created!


In [ ]:
pl_model.model = model
model.config.decoder_start_token_id = pl_model.tokenizer.cls_token_id
model.config.pad_token_id = pl_model.tokenizer.pad_token_id


In [ ]:
trainer = pl.Trainer(
    max_epochs=1,
    devices=1,
    callbacks=[
        EarlyStopping(monitor="val_loss", patience=5, mode="min"),
        TQDMProgressBar(),
    ],
    default_root_dir="/data/home/ngjhn/read",
    num_sanity_val_steps=0,
    check_val_every_n_epoch=2,
)
trainer.fit(pl_model, data_module)


/data/home/ngjhn/miniconda3/envs/awsnccl/lib/python3.8/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /data/home/ngjhn/.local/lib/python3.8/site-packages/ ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


loading annotations into memory...
Done (t=0.56s)
creating index...
index created!
loading annotations into memory...
Done (t=0.26s)
creating index...
index created!


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type                      | Params
--------------------------------------------------------
0 | model     | VisionEncoderDecoderModel | 224 M 
1 | val_loss  | AvgMetric                 | 0     
2 | test_loss | AvgMetric                 | 0     
--------------------------------------------------------
138 M     Trainable params
86.4 M    Non-trainable params
224 M     Total params
898.261   Total estimated model params size (MB)


Epoch 0:   0%|          | 9/2587 [00:06<30:15,  1.42it/s, v_num=26, loss=nan.0]   

/data/home/ngjhn/miniconda3/envs/awsnccl/lib/python3.8/site-packages/pytorch_lightning/trainer/call.py:54: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


Process Process-32:
Process Process-31:
Process Process-30:
Process Process-33:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/data/home/ngjhn/miniconda3/envs/awsnccl/lib/python3.8/multiprocessing/process.py", line 318, in _bootstrap
    util._exit_function()
  File "/data/home/ngjhn/miniconda3/envs/awsnccl/lib/python3.8/multiprocessing/process.py", line 318, in _bootstrap
    util._exit_function()
Traceback (most recent call last):
  File "/data/home/ngjhn/miniconda3/envs/awsnccl/lib/python3.8/multiprocessing/process.py", line 318, in _bootstrap
    util._exit_function()
  File "/data/home/ngjhn/miniconda3/envs/awsnccl/lib/python3.8/multiprocessing/util.py", line 360, in _exit_function
    _run_finalizers()
  File "/data/home/ngjhn/miniconda3/envs/awsnccl/lib/python3.8/multiprocessing/util.py", line 360, in _exit_function
    _run_finalizers()
  File "/data/home/ngjhn/miniconda3/envs/awsnccl/lib/python3.8/multiprocessi